# End-to-End Diabetes Machine Learning Pipeline II


In [16]:
# Helper Functions 
#utils.py 
#helpers.py -> dışarıdan çağırılabilir bunlar fonksiyonları bir arada görmemizi sağlar   
# config.py -> dışarıdan ayarlanabilir şeyleri bu dosya içerisine de koyabiliriz. 
# from ile çağırabilir ve kullanabilir  

In [3]:
import joblib
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

In [4]:
df = pd.read_csv("C:/Users/tolga/OneDrive/Masaüstü/ML_YAZ_KAMPI/Machine Learning/datasets/diabetes.csv")

In [5]:
# grab_col_names 

def grab_col_names(dataframe, cat_th=10, car_th=20):
    """
    Veri setindeki kategorik, numerik ve kategorik fakat kardinal değişkenlerin isimlerini verir.
    Not: Kategorik değişkenlerin içerisine numerik görünümlü kategorik değişkenler de dahildir.
    Parameters
    ------
        dataframe: dataframe
                Değişken isimleri alınmak istenilen dataframe
        cat_th: int, optional
                numerik fakat kategorik olan değişkenler için sınıf eşik değeri
        car_th: int, optinal
                kategorik fakat kardinal değişkenler için sınıf eşik değeri
    Returns
    ------
        cat_cols: list
                Kategorik değişken listesi
        num_cols: list
                Numerik değişken listesi
        cat_but_car: list
                Kategorik görünümlü kardinal değişken listesi
    Examples
    ------
        import seaborn as sns
        df = sns.load_dataset("iris")
        print(grab_col_names(df))
    Notes
    ------
        cat_cols + num_cols + cat_but_car = toplam değişken sayısı
        num_but_cat cat_cols'un içerisinde.
        Return olan 3 liste toplamı toplam değişken sayısına eşittir: cat_cols + num_cols + cat_but_car = değişken sayısı
    """

    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    return cat_cols, num_cols, cat_but_car

In [6]:
#görevi : kendisine gelen değişkenleri alt ve üst sınır tanımlar 
# ön tanımlı değerleri : q1: 0.25 , q3: 0.75
def outlier_thresholds(dataframe, col_name, q1=0.25, q3=0.75):
    quartile1 = dataframe[col_name].quantile(q1)
    quartile3 = dataframe[col_name].quantile(q3)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

In [7]:
# aykırı değeri gönder yerine bunu koy dersek kullanırız

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [8]:
# yeni türettiğimiz değişkenlerde kategorik değişkenleri ortaya çıkarmak için 
# dummy değişken tuzağından kurtulmak istiyoruz 

def one_hot_encoder(dataframe, categorical_cols, drop_first=False):
    dataframe = pd.get_dummies(dataframe, columns=categorical_cols, drop_first=drop_first)
    return dataframe

In [9]:
# hepsini tek bir fonksiyon içerisinde de yazabiliriz. 

def diabetes_data_prep(dataframe):
    dataframe.columns = [col.upper() for col in dataframe.columns]

    # Glucose
    dataframe['NEW_GLUCOSE_CAT'] = pd.cut(x=dataframe['GLUCOSE'], bins=[-1, 139, 200], labels=["normal", "prediabetes"])

    # Age
    dataframe.loc[(dataframe['AGE'] < 35), "NEW_AGE_CAT"] = 'young'
    dataframe.loc[(dataframe['AGE'] >= 35) & (dataframe['AGE'] <= 55), "NEW_AGE_CAT"] = 'middleage'
    dataframe.loc[(dataframe['AGE'] > 55), "NEW_AGE_CAT"] = 'old'

    # BMI
    dataframe['NEW_BMI_RANGE'] = pd.cut(x=dataframe['BMI'], bins=[-1, 18.5, 24.9, 29.9, 100],
                                        labels=["underweight", "healty", "overweight", "obese"])

    # BloodPressure
    dataframe['NEW_BLOODPRESSURE'] = pd.cut(x=dataframe['BLOODPRESSURE'], bins=[-1, 79, 89, 123],
                                            labels=["normal", "hs1", "hs2"])

    cat_cols, num_cols, cat_but_car = grab_col_names(dataframe, cat_th=5, car_th=20)

    cat_cols = [col for col in cat_cols if "OUTCOME" not in col]

    df = one_hot_encoder(dataframe, cat_cols, drop_first=True)

    df.columns = [col.upper() for col in df.columns]

    cat_cols, num_cols, cat_but_car = grab_col_names(df, cat_th=5, car_th=20)

    cat_cols = [col for col in cat_cols if "OUTCOME" not in col]

    replace_with_thresholds(df, "INSULIN")

    X_scaled = StandardScaler().fit_transform(df[num_cols])
    df[num_cols] = pd.DataFrame(X_scaled, columns=df[num_cols].columns)

    y = df["OUTCOME"]
    X = df.drop(["OUTCOME"], axis=1)

    return X, y

In [10]:
def base_models(X,y, scoring = "roc_auc"): 
    print("Base Models.....") 
    classifiers = [('LR',LogisticRegression()), 
                  ('KNN',KNeighborsClassifier()), 
                  ("SVC",SVC()), 
                  ("CART",DecisionTreeClassifier()), 
                  ("RF",RandomForestClassifier()),
                  ('Adaboost',AdaBoostClassifier()), 
                  ('GBM',GradientBoostingClassifier()), 
                  ('XGBoost',XGBClassifier(use_label_encoder=False,eval_metric='logloss')), 
                  ('LightGBM', LGBMClassifier()), 
                  #('CatBoost', CatBoostClassifier(verbose= False)) 
                  ] 
    
    for name, classifier in classifiers:
        cv_results = cross_validate(classifier, X, y, cv=3, scoring=scoring)
        print(f"{scoring}: {round(cv_results['test_score'].mean(), 4)} ({name}) ")

In [11]:
knn_params = {"n_neighbors": range(2, 50)}

cart_params = {'max_depth': range(1, 20),
               "min_samples_split": range(2, 30)}

rf_params = {"max_depth": [8, 15, None],
             "max_features": [5, 7, "auto"],
             "min_samples_split": [15, 20],
             "n_estimators": [200, 300]}

xgboost_params = {"learning_rate": [0.1, 0.01],
                  "max_depth": [5, 8],
                  "n_estimators": [100, 200]}

lightgbm_params = {"learning_rate": [0.01, 0.1],
                   "n_estimators": [300, 500]}

In [12]:
classifiers = [('KNN', KNeighborsClassifier(), knn_params),
               ("CART", DecisionTreeClassifier(), cart_params),
               ("RF", RandomForestClassifier(), rf_params),
               ('XGBoost', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), xgboost_params),
               ('LightGBM', LGBMClassifier(), lightgbm_params)]

In [13]:
def hyperparameter_optimization(X,y, cv=3, scoring= "roc_auc"): 
    print("Hyperparameter Optimization...") 
    best_models = {} 
    for name,classifier,params in classifiers: 
        print(f"#######  {name} ########")  
        cv_results = cross_validate(classifier, X,y, cv=cv, scoring=scoring) 
        print(f"{scoring} (Before):{round(cv_results['test_score'].mean(),4)}") 
        # before : hyperparametre öncesi skoru ekrana yazdırılıyor
        gs_best = GridSearchCV(classifier,params,cv=cv,n_jobs=-1,verbose=False).fit(X,y) 
        final_model = classifier.set_params(**gs_best.best_params_) 
        # after : hyperparametre sonrası skoru yazdırır 
        cv_results = cross_validate(final_model,X,y, cv=cv, scoring=scoring) 
        print(f"{scoring} (After):{round(cv_results['test_score'].mean(),4)}") 
        print(f"{name} best params:{gs_best.best_params_}",end="\n\n") 
        best_models[name] = final_model 
    return best_models

In [14]:
def voting_classifier(best_models, X, y): 
    print("Voting Classifier....") 
    # bana kullanacak olduğun modelleri söyle 
    # bu 3 modele göre en iyi modeli ortaya çıkar 
    voting_clf = VotingClassifier(estimators=[('KNN',best_models["KNN"]),('RF',best_models["RF"]),
                                             ('LightGBM',best_models["LightGBM"])],
                                 voting='soft').fit(X,y) 
    
    cv_results = cross_validate(voting_clf, X, y, cv=3, scoring=["accuracy","f1","roc_auc"]) 
    print(f"Accuracy: {cv_results['test_accuracy'].mean()}") 
    print(f"F1Score: {cv_results['test_f1'].mean()}") 
    print(f"ROC_AUC:{cv_results['test_roc_auc'].mean()}") 
    return voting_clf

In [17]:
# işletim seviyesi şeklinde çalıştırıyor olmam gerekiyor  
# Pipeline Main Function 
# terminalden bunu çalıştırmak istediğimde bu faydalı bir kullanımdır  
# dışardan işlemi değiştirmek isterse main kısmından halledebilir 
# terminale gelip açmak için ; "python diabetes_pipeline.py" yazınız!!!!
# #######################################################################
def main():  
    # veriyi çek
    df = pd.read_csv("C:/Users/tolga/OneDrive/Masaüstü/ML_YAZ_KAMPI/Machine Learning/datasets/diabetes.csv")
    # veriyi önişleme
    X,y = diabetes_data_prep(df)  
    # genel modellere bak
    base_models(X,y) 
    # hyperparametre optimizasyonu 
    best_models = hyperparameter_optimization(X,y)  
    # en iyi modeli seç
    voting_clf = voting_classifier(best_models, X,y)  
    # model sürecini kaydet
    joblib.dump(voting_clf,"voting_clf.pkl") 
    return voting_clf 
# double under score 
if __name__ == "__main__":  
    print("işlem başladı....")
    main()

işlem başladı....
Observations: 768
Variables: 13
cat_cols: 5
num_cols: 8
cat_but_car: 0
num_but_cat: 4
Observations: 768
Variables: 17
cat_cols: 9
num_cols: 8
cat_but_car: 0
num_but_cat: 9
Base Models.....
roc_auc: 0.8409 (LR) 
roc_auc: 0.791 (KNN) 
roc_auc: 0.8355 (SVC) 
roc_auc: 0.6517 (CART) 
roc_auc: 0.8268 (RF) 
roc_auc: 0.8196 (Adaboost) 
roc_auc: 0.8248 (GBM) 
roc_auc: 0.8015 (XGBoost) 
roc_auc: 0.807 (LightGBM) 
Hyperparameter Optimization...
#######  KNN ########
roc_auc (Before):0.8211
roc_auc (After):0.8211
KNN best params:{'n_neighbors': 20}

#######  CART ########
roc_auc (Before):0.7943
roc_auc (After):0.7943
CART best params:{'max_depth': 6, 'min_samples_split': 23}

#######  RF ########
roc_auc (Before):0.8313
roc_auc (After):0.8355
RF best params:{'max_depth': 8, 'max_features': 7, 'min_samples_split': 15, 'n_estimators': 200}

#######  XGBoost ########
roc_auc (Before):0.8179
roc_auc (After):0.8179
XGBoost best params:{'learning_rate': 0.01, 'max_depth': 5, 'n_estima

In [ ]:
# git github 
# makefile -> code otomasyon aracı  -> make train 
# veri tabanları 
# log 
# class 
# docker 
# requirement.txt 